In [5]:
from astropy.io import fits
from search_data import search_candidates
import scipy

In [6]:
event_file = 'acisf08490N004_evt2.fits'
src_file = 's3_expmap_src.fits'

In [9]:
# remake of window calculation
with fits.open(event_file) as hdul:
    information = hdul[1].header
    obs = information['OBS_ID']
    t_start = information['TSTART']
    t_stop = information['TSTOP']

time = t_stop - t_start
window = 20

def print_start_end_times(time, t_start, t_end, window):
    time /= 1000
    N_regions = int(time // window)
    residual = time - N_regions * window
    array = []

    print(f"Time: {time} s")
    print(f"Window: {window} s")
    print(f"Number of regions: {N_regions}")
    print(f"Residual: {residual} s")

    print()
    print(f"... PASS")
    print(f"\ti: index, t_1: t_1 ((t_1 - t_start) / 1000), t_2: t_2 ((t_2 - t_start) / 1000")
    print()

    # forward pass
    print("FORWARD PASS")
    for i in range(N_regions):
        t_1, t_2 = window * 1000 * i + t_start, window * 1000 * (i + 1) + t_start
        print(f"\ti: {i}, t_1: {t_1} ({(t_1 - t_start) / 1000}), t_2: {t_2} ({(t_2 - t_start) / 1000})")
        array.append(t_2 - t_1)
    
    t_1, t_2 = window * 1000 * (i + 1) + t_start, window * 1000 * (i + 1) + t_start + residual * 1000
    print(
        f"\ti: {i}, t_1: {t_1} ({(t_1 - t_start) / 1000}), t_2: {t_2} ({(t_2 - t_start) / 1000})")
    array.append(t_2 - t_1)

    # backward pass
    print("BACKWARD PASS")
    for i in range(len(array)):
        if i == 0:
            t_1, t_2 = 0, array[-1]
            print(
                f"\ti: {i}, t_1: {t_1} ({(t_1 - t_start) / 1000}), t_2: {t_2} ({(t_2 - t_start) / 1000})")
        else:
            t_1, t_2 = array[-1] + window * (i - 1) * 1000, array[-1] + window * i * 1000
            print(
                f"\ti: {i}, t_1: {t_1} ({(t_1 - t_start) / 1000}), t_2: {t_2} ({(t_2 - t_start) / 1000})")
        
    # shift
    print("SHIFTED PASS")
    a = 0.0
    for i in range(len(array)):
        a += array[i]
        t_1, t_2 = a - window/2 * 1000, a + window/2 * 1000
        print(f"\ti: {i}, t_1: {t_1} ({(t_1 - t_start) / 1000}), t_2: {t_2} ({(t_2 - t_start) / 1000})")

print_start_end_times(time, t_start, t_stop, window)

Time: 97.17359215003252 s
Window: 20 s
Number of regions: 4
Residual: 17.17359215003252 s

... PASS
	i: index, t_1: t_1 ((t_1 - t_start) / 1000), t_2: t_2 ((t_2 - t_start) / 1000

FORWARD PASS
	i: 0, t_1: 296877766.93042 (0.0), t_2: 296897766.93042 (20.0)
	i: 1, t_1: 296897766.93042 (20.0), t_2: 296917766.93042 (40.0)
	i: 2, t_1: 296917766.93042 (40.0), t_2: 296937766.93042 (60.0)
	i: 3, t_1: 296937766.93042 (60.0), t_2: 296957766.93042 (80.0)
	i: 3, t_1: 296957766.93042 (80.0), t_2: 296974940.52257 (97.17359215003252)
BACKWARD PASS
	i: 0, t_1: 0 (-296877.76693042), t_2: 17173.59215003252 (-296860.59333827)
	i: 1, t_1: 17173.59215003252 (-296860.59333827), t_2: 37173.59215003252 (-296840.59333827)
	i: 2, t_1: 37173.59215003252 (-296840.59333827), t_2: 57173.59215003252 (-296820.59333827)
	i: 3, t_1: 57173.59215003252 (-296820.59333827), t_2: 77173.59215003252 (-296800.59333827)
	i: 4, t_1: 77173.59215003252 (-296800.59333827), t_2: 97173.59215003252 (-296780.59333827)
SHIFTED PASS
	i: 

In [8]:
search_candidates(src_file, event_file)

8490
acisf08490N004_evt2.fits
0.0 20000.0 17.17359215003252 4
20000.0 40000.0 17.17359215003252 4
40000.0 60000.0 17.17359215003252 4
60000.0 80000.0 17.17359215003252 4
80000.0 97173.59215003252 17.17359215003252 4
[20000.0, 20000.0, 20000.0, 20000.0, 17173.59215003252]
0 17173.59215003252 17.17359215003252 4
no events
17173.59215003252 37173.59215003252 17.17359215003252 4
no events
37173.59215003252 57173.59215003252 17.17359215003252 4
no events
57173.59215003252 77173.59215003252 17.17359215003252 4
no events
77173.59215003252 97173.59215003252 17.17359215003252 4
no events
10000.0 30000.0
no events
30000.0 50000.0
no events
50000.0 70000.0
no events
70000.0 90000.0
no events
87173.59215003252 107173.59215003252
no events
8490


c:\Users\jesse\Desktop\Stage\code\Code CXO\search_data.py:360: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  myFinalList = pd.unique(candidates).tolist()
